<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Scraping/AmazonScrapeProducts4Category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install python-dateutil lxml requests selectorlib

In [ ]:
#IMPORT E FUNZIONI DI SUPPORTO
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
#from clarifai.rest import ClarifaiApp
#from clarifai.rest import ApiError
import numpy as np
import time
import json
import csv
import random
import pandas as pd
import requests
from lxml import html
import shutil
import os
import json
import datetime
import math
import io
import math
from google.colab import files
from selectorlib import Extractor
import requests 
from time import sleep
from dateutil import parser as dateparser


# AUTHENTICATE IN GOOGLE DRIVE
def authenticate():
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  return drive
drive = authenticate()


def create_folder_in_drive(gdrive, folder_name, parent_folder_id):
  folder_metadata = {'title': folder_name,'mimeType': 'application/vnd.google-apps.folder',
                    'parents': [{"kind": "drive#fileLink", "id": parent_folder_id}]
                    }
  folder = gdrive.CreateFile(folder_metadata)
  folder.Upload()
  print(folder)
  # Return folder informations
  print('title: %s, id: %s' % (folder['title'], folder['id']))
  return folder['id']


def drop_unnamed(df):
  cols = [c for c in df.columns if c.lower()[:7] != 'unnamed']
  return df[cols]


def upload_file(filename, folder_id):
  drive = authenticate()
  fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
  drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
  # Check if file already exists in Google Drive (prevents duplicates)
  for file in fileList:
      if file['title'] == filename:  # The file already exists, then overwrite it
          fileID = file['id']
          drive_file = drive.CreateFile({'id': fileID, 'title': filename, 'parents': [{'id': folder_id}]})
  # Upload user picture on Google Drive
  drive_file.SetContentFile(filename)  # path of local file content
  drive_file.Upload()  # Upload the file.
  return drive_file['id']


def downloadUser(business_id, user_id, photo_folder, counter):
    authenticate()
    
    filename = user_id + '.jpg'
    url = 'https://www.yelp.com/user_details?userid=' + user_id
    folder_id = photo_folder

    # CHECK DUPLICATE
    fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
    exists = False
    # Check if file already exists in Google Drive (prevents duplicates)
    for file in fileList:
        if file['title'] == filename:  # The file already exists
            exists = True
    
    if not exists:
      try:
        # Find user picture from web page
        page_content = requests.get(url)
        page_content.raise_for_status()
        print(page_content)
        tree = html.fromstring(page_content.content)
        if len(tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')) == 0:
          print('PICTURE NOT FOUND of user '+ user_id)
        else: # Picture found
          image_url = tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')[0]

          drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
          
          response = requests.get(image_url, stream=True)
          
          # Create a local copy of user picture
          with open(filename, 'wb') as out_file:
              shutil.copyfileobj(response.raw, out_file)
          del response

          # Upload user picture on Google Drive
          drive_file.SetContentFile(filename)
          drive_file.Upload()
          if os.path.exists(filename):
              os.remove(filename)
          else:
              print("The file does not exist")
          print(user_id + ' downloaded')
      except:
        #code = page_content.status_code
        #if code==503:
        #  print("ERROR 503")
        #if code==404:
        #  print("User not found")
        print("Error")

def downloadUserWithProxy(business_id, user_id, photo_folder, counter, ip_addresses):
    authenticate()
    
    filename = user_id + '.jpg'
    url = 'https://www.yelp.com/user_details?userid=' + user_id
    folder_id = photo_folder

    # CHECK DUPLICATE
    fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
    exists = False
    # Check if file already exists in Google Drive (prevents duplicates)
    for file in fileList:
        if file['title'] == filename:  # The file already exists
            exists = True
    
    if not exists:
      downloaded = False
      while len(ip_addresses) != 0 and not downloaded:
        proxy_index = 0
        proxy = {"http": ip_addresses[proxy_index], "https": ip_addresses[proxy_index]}
        try:
          # Check the proxy address
          #response = requests.get('https://httpbin.org/ip',proxies=proxy, timeout=5)
          #print(response.json())
          
          # Find user picture from web page
          page_content = requests.get(url, proxies=proxy, timeout=10)
          print(page_content)
          if page_content.status_code != 503:
            tree = html.fromstring(page_content.content)
            if len(tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')) == 0:
              print('PICTURE NOT FOUND of user '+ user_id)
            else: # Picture found
              image_url = tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')[0]

              drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
              
              response = requests.get(image_url, proxies=proxy, timeout=30, stream=True)
              
              # Create a local copy of user picture
              with open(filename, 'wb') as out_file:
                  shutil.copyfileobj(response.raw, out_file)
              del response

              # Upload user picture on Google Drive
              drive_file.SetContentFile(filename)
              drive_file.Upload()
              if os.path.exists(filename):
                  os.remove(filename)
              else:
                  print("The file does not exist")
              print(user_id + ' downloaded')
              downloaded = True
              ip_addresses.insert(0, ip_addresses.pop(proxy_index))
          else:
            print("Skipping. Connnection error")
            del ip_addresses[proxy_index]
            print("Proxy rimanenti: ")
            print(ip_addresses)
            #restart_runtime()
        except Exception as e:
          print(e)
          print("Skipping. Connnection error")
          del ip_addresses[proxy_index]
          print("Proxy rimanenti: ")
          print(ip_addresses)
      if len(ip_addresses)==0:
        print("Nessun proxy disponibile")


def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = html.fromstring(response.content)
    proxies = []
    for i in parser.xpath('//tbody/tr')[:500]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.append(proxy)
    print(proxies)
    proxies.append('18.223.213.237:3838')
    proxies.append('18.223.103.221:3838')
    proxies.append('117.69.230.23:9999')
    with open('http_proxies.txt', 'w') as f:
      for item in proxies:
        f.write("%s\n" % item)
    return proxies

def scrape(e, url):    
  headers = {
      'authority': 'www.amazon.com',
      'pragma': 'no-cache',
      'cache-control': 'no-cache',
      'dnt': '1',
      'upgrade-insecure-requests': '1',
      'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
      'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'sec-fetch-site': 'none',
      'sec-fetch-mode': 'navigate',
      'sec-fetch-dest': 'document',
      'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
  }

  # Download the page using requests
  print("Downloading %s"%url)
  r = requests.get(url, headers=headers)
  # Simple check to check if page was blocked (Usually 503)
  if r.status_code > 500:
      if "To discuss automated access to Amazon data please contact" in r.text:
          print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
      else:
          print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
      return None
  # Pass the HTML of the page and create 
  return e.extract(r.text)


In [ ]:
#carico i file YML
download = drive.CreateFile({'id': '12DmvHhjrBHI8rolxkkIeLYF6_ZynuUxT'})
download.GetContentFile('AmazonReviews.yml')

download2 = drive.CreateFile({'id': '1PuYdOat85pFtE1XC6__WIxeIe66BQuEp'})
download2.GetContentFile('AmazonProducts.yml')

download3 = drive.CreateFile({'id': '1s2JwxZqTHALk1VKqpe_shwMxLUIhKP-V'})
download3.GetContentFile('CategorySearch.yml')

download4 = drive.CreateFile({'id': '1YKZmpUdqKzCA2xOgYTgRU_QGso3NG1Xx'})
download4.GetContentFile('AmazonUserPage.yml')

# Create Extractors by reading from the YAML file
userExtractor = Extractor.from_yaml_file('AmazonUserPage.yml')
reviewsExtractor = Extractor.from_yaml_file('AmazonReviews.yml')
productExtractor = Extractor.from_yaml_file('AmazonProducts.yml')
categoryExtractor = Extractor.from_yaml_file('CategorySearch.yml') 

#inizializza queries
download = drive.CreateFile({'id': '1V-L7IqaPVcBp4RvWKnUXt00SHrsT1e9O'})
download.GetContentFile('queries.csv')

queries = pd.read_csv("queries.csv")
queries = drop_unnamed(queries)

for i, x in queries.iterrows():
  if x["products_gtree"]==-1:
    df = pd.DataFrame(columns=["position","asin","url"])
    df.to_csv(str(x["cardinal"])+ "_gtree.csv")
    id_new_folder = create_folder_in_drive(drive, str(x["cardinal"]), '13N3YrLs4LD8i4uZ5JJqrkEVCVUPq1Yje')
    id_new_folder2 = create_folder_in_drive(drive, "reviews_folder", id_new_folder)
    id_new_file = upload_file(str(x["cardinal"])+ "_gtree.csv",id_new_folder)
    id_new_folder3 = create_folder_in_drive(drive, "fairness_data", id_new_folder)
    id_new_folder4 = create_folder_in_drive(drive, "face_data", id_new_folder)
    queries.loc[queries["cardinal"]==x["cardinal"], 'products_folder'] = id_new_folder
    queries.loc[queries["cardinal"]==x["cardinal"], 'products_gtree'] = id_new_file
    queries.loc[queries["cardinal"]==x["cardinal"], 'reviews_folder'] = id_new_folder2
    queries.loc[queries["cardinal"]==x["cardinal"], 'fairness_data'] = id_new_folder3
    queries.loc[queries["cardinal"]==x["cardinal"], 'face_data'] = id_new_folder4
    queries = drop_unnamed(queries)
    queries_csv = queries.to_csv("queries.csv")
    upload_file("queries.csv","1BJCILjUPI5gUIdkgT7nWwU_uz0VZhY7V")

In [ ]:
query=1
products_gtree_id = queries.loc[queries["cardinal"]==query,"products_gtree"].tolist()[0]
products_folder_id = queries.loc[queries["cardinal"]==query,"products_folder"].tolist()[0]
reviews_folder_id = queries.loc[queries["cardinal"]==query,"reviews_folder"].tolist()[0]
fairness_data_id = queries.loc[queries["cardinal"]==query,"fairness_data"].tolist()[0]
face_data_id = queries.loc[queries["cardinal"]==query,"face_data"].tolist()[0]

download = drive.CreateFile({'id': products_gtree_id})
download.GetContentFile(str(query)+'_gtree.csv')

gtree = pd.read_csv(str(query)+'_gtree.csv')
gtree = drop_unnamed(gtree)

In [ ]:
gtree

In [ ]:
#MANUAL SCRAPE REVIEWS FOR PRODUCT

asinList = gtree['asin'].tolist()
for asin in asinList:
  
  drive = authenticate()
  for counter in range(300):
    print(counter)

    gtree = pd.read_csv(str(query)+'_gtree.csv')
    nextPage = gtree.loc[gtree['asin']==asin,"nextPage"].tolist()[0]
    product_url = gtree.loc[gtree['asin']==asin,"url"].tolist()[0]
    reviews_data_gtree_id = gtree.loc[gtree['asin']==asin,"reviews_data"].tolist()[0]

    if not nextPage =='-1':

      reviews_df = pd.DataFrame(columns=["name","profileUrl","reviewText","voteText","dateText"])

      if reviews_data_gtree_id=='-1' or reviews_data_gtree_id==-1:
        review_csv = reviews_df.to_csv(asin+"_reviews.csv")
        file_id = upload_file(asin+"_reviews.csv", reviews_folder_id)
        gtree.loc[gtree['asin']==asin,"reviews_data"] = file_id
        gtree = drop_unnamed(gtree)
        gtree.to_csv(str(query)+'_gtree.csv')  
        upload_file(str(query)+'_gtree.csv',products_folder_id)
      else:
        download = drive.CreateFile({'id': reviews_data_gtree_id})
        download.GetContentFile(asin+"_reviews.csv")

        reviews_df = pd.read_csv(asin+"_reviews.csv")
        reviews_df = drop_unnamed(reviews_df)

      if product_url == nextPage: #inizia da capo
        nextPage = "https://www.amazon.com/product-reviews/"+asin+"/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
        temp_data=scrape(reviewsExtractor, nextPage)
      else: #riprende da tot reviews
        temp_data=scrape(reviewsExtractor, nextPage)

      if 'nextPage' in temp_data.keys():
        if temp_data['nextPage'] is not None:
          gtree = pd.read_csv(str(query)+'_gtree.csv')
          gtree.loc[gtree['asin']==asin,'nextPage'] = "https://www.amazon.com" + str(temp_data['nextPage'])
          gtree = drop_unnamed(gtree)
          gtree.to_csv(str(query)+'_gtree.csv')
          upload_file(str(query)+'_gtree.csv',products_folder_id)
      print(temp_data)

      while 'nextPage' in temp_data.keys():
        print("nextPage")
        print(temp_data['nextPage'])
        if temp_data['reviews'] is not None:
          for review in temp_data['reviews']:
            new_row = {
                "name":review["name"],
                "profileUrl":review["profileUrl"],
                "reviewText":review["reviewText"],
                "voteText":review["voteText"],
                "dateText":review["dateText"]
            }
            reviews_df = reviews_df.append(new_row,ignore_index=True)
          reviews_df = drop_unnamed(reviews_df)
          reviews_df.to_csv(asin+"_reviews.csv")
          upload_file(asin+"_reviews.csv", reviews_folder_id)

          if temp_data['nextPage'] is not None:#forse puoi portarlo fuori dal primo if
            try:
              nextPage_data = scrape(reviewsExtractor, "https://www.amazon.com" + str(temp_data['nextPage']))
              print(nextPage_data)
              gtree.loc[gtree['asin']==asin,'nextPage'] = "https://www.amazon.com" + temp_data['nextPage']
              #gtree = drop_unnamed(gtree)
              #gtree.to_csv(str(query)+'_gtree.csv')
              #upload_file(str(query)+'_gtree.csv',products_folder_id)
            except:
              gtree.loc[gtree['asin']==asin,'nextPage'] = "https://www.amazon.com" + temp_data['nextPage']
              gtree = drop_unnamed(gtree)
              gtree.to_csv(str(query)+'_gtree.csv')
              upload_file(str(query)+'_gtree.csv',products_folder_id)
              nextPage_data={}
              print("CANT SCRAPE NEXT PAGE")
          try:
            pageNumber = temp_data['nextPage'].split("pageNumber=")[1][0:3]
            if pageNumber == str(501):
              print("!!!501!!!")
              gtree.loc[gtree['asin']==asin,'nextPage'] = -1
              gtree = drop_unnamed(gtree)
              gtree.to_csv(str(query)+'_gtree.csv')
              upload_file(str(query)+'_gtree.csv',products_folder_id)
          except:
            print("boh")
        ###########################################################
        if 'end' in temp_data.keys():
          if temp_data['end'] is not None:
            gtree.loc[gtree['asin']==asin,'nextPage'] = -1
            gtree = drop_unnamed(gtree)
            gtree.to_csv(str(query)+'_gtree.csv')
            upload_file(str(query)+'_gtree.csv',products_folder_id)
        ###########################################################
        if temp_data['nextPage'] is None:
          nextPage_data={}
        temp_data = nextPage_data
        sleep(5)
      gtree = drop_unnamed(gtree)
      gtree.to_csv(str(query)+'_gtree.csv')
      upload_file(str(query)+'_gtree.csv',products_folder_id)
      print("extraction completed for "+asin)
    else:
      print(asin + " alredy completed")
print("END END END")

In [ ]:
#SEGNA ASIN COME COMPLETATO
query=1
asin=""#inserire asin completato

download = drive.CreateFile({'id': '1V-L7IqaPVcBp4RvWKnUXt00SHrsT1e9O'})
download.GetContentFile('queries.csv')

queries = pd.read_csv("queries.csv")
queries = drop_unnamed(queries)

products_gtree_id = queries.loc[queries["cardinal"]==query,"products_gtree"].tolist()[0]
products_folder_id = queries.loc[queries["cardinal"]==query,"products_folder"].tolist()[0]
reviews_folder_id = queries.loc[queries["cardinal"]==query,"reviews_folder"].tolist()[0]

download = drive.CreateFile({'id': products_gtree_id})
download.GetContentFile(str(query)+'_gtree.csv')

gtree = pd.read_csv(str(query)+'_gtree.csv')
gtree = drop_unnamed(gtree)

gtree.loc[gtree["asin"]==asin,"nextPage"]='-1'
gtree = drop_unnamed(gtree)
gtree.to_csv(str(query)+'_gtree.csv')
upload_file(str(query)+'_gtree.csv',products_folder_id)

'1ptG24HpoP5Nj5amLD7-VkYDJRxLBD9et'